# Battle of Neighborhoods

In [7]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import sys
!{sys.executable} -m pip install geocoder

     |████████████████████████████████| 102kB 10.9MB/s ta 0:00:01
     |████████████████████████████████| 92kB 6.2MB/s eta 0:00:011
     |████████████████████████████████| 829kB 7.3MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future


## Introduction

### Background

Barcelona is recognized as a global city for its cultural, touristic and financial importance. There are more than 3.5 million of tourist coming to the city each year. But, apart from going for a holiday, there is also a large number of people that move to this city to look for job opportunities. 

## Business problem

From my experience living in this city, I have realized that there is a large community of italians in Barcelona. From my point of view, this can be due to the similarity between spanish and italian culture, what makes attractive to this group of people to move to this catalan city. Nevertheless, I am sure that some italians will prefer to live in a neighborhood with a strong presence of italian culture. For this reason, I though that it could be useful to study which borough and which neighborhood has a stronger presence of italian culture. To do so, I will analyze the number of italian restaurants in each place, and from this data, I will derive the conclusion.

## Acquisition of data

1. The database used is from the Catalan gouvernment page. It contains the name of the Boroughs of Barcelona as well as its neighborhoods. Nevertheless, I didn't find a database with the altitude and latitude included so I will have to add this data after. 

In [3]:
filename = 'districtes_i_barris_170705.csv'
#Read csv
pdf = pd.read_csv(filename)
print ("Shape of dataset: ", pdf.shape)
pdf.head()

Shape of dataset:  (73, 4)


,CODI_DISTRICTE,NOM_DISTRICTE,CODI_BARRI,NOM_BARRI
0,1,Ciutat Vella,1,el Raval
1,1,Ciutat Vella,2,el Barri Gòtic
2,1,Ciutat Vella,3,la Barceloneta
3,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera"
4,2,Eixample,5,el Fort Pienc


The database is in Catalan so I will translate the features to english.

In [4]:
pdf.columns=['Borough_ID', 'Borough', 'Neighborhood_ID','Neighborhood']
pdf["Latitude"]=""
pdf["Longitude"]=""
pdf.head()

,Borough_ID,Borough,Neighborhood_ID,Neighborhood,Latitude,Longitude
0,1,Ciutat Vella,1,el Raval,,
1,1,Ciutat Vella,2,el Barri Gòtic,,
2,1,Ciutat Vella,3,la Barceloneta,,
3,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",,
4,2,Eixample,5,el Fort Pienc,,


I drop the name of neighborhoods that were redundant and the columns that were useless for our problem.

In [6]:
pdf=pdf.drop(['Borough_ID', 'Neighborhood_ID'], axis=1)
pdf.Neighborhood[3]='Sant Pere'
pdf.Neighborhood[21]='Vallvidrera'
pdf

,Borough,Neighborhood,Latitude,Longitude
0,Ciutat Vella,el Raval,,
1,Ciutat Vella,el Barri Gòtic,,
2,Ciutat Vella,la Barceloneta,,
3,Ciutat Vella,Sant Pere,,
4,Eixample,el Fort Pienc,,
...,...,...,...,...
68,Sant Martí,Diagonal Mar i el Front Marítim del Poblenou,,
69,Sant Martí,el Besòs i el Maresme,,
70,Sant Martí,Provençals del Poblenou,,
71,Sant Martí,Sant Martí de Provençals,,


I add the altitude and longitude of each neighborhood using a geocode

In [8]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim()
city ="Barcelona"
country ="Spain"
data=0
for data in range (73):
    borough1=pdf.Borough[data]
    neighborhood1=pdf.Neighborhood[data]
    loc = geolocator.geocode(neighborhood1 + ','+ borough1 +','+ city+','+ country)
    pdf.Latitude[data]= loc.latitude 
    pdf.Longitude[data]=loc.longitude
pdf

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


,Borough,Neighborhood,Latitude,Longitude
0,Ciutat Vella,el Raval,41.3795,2.16837
1,Ciutat Vella,el Barri Gòtic,41.3834,2.17691
2,Ciutat Vella,la Barceloneta,41.3807,2.18993
3,Ciutat Vella,Sant Pere,41.3883,2.17741
4,Eixample,el Fort Pienc,41.3959,2.18232
...,...,...,...,...
68,Sant Martí,Diagonal Mar i el Front Marítim del Poblenou,41.4032,2.21338
69,Sant Martí,el Besòs i el Maresme,41.415,2.21388
70,Sant Martí,Provençals del Poblenou,41.4124,2.20489
71,Sant Martí,Sant Martí de Provençals,41.4165,2.19897


2. The second source of data will be Foursquare to find the nearby venues of each neighborhood in ordt